In [65]:
import requests
import pandas as pd
import numpy as np
import matplotlib as plt
import tqdm
import datetime
import json
import collections
%matplotlib inline

#1.	Выгрузить описания открытых вакансий с любой из площадок (hh.ru, rabota.sber.ru, indeed.com)

In [66]:
sber = '3529'
page = 1
num_per_page = 100
Samara = 1

In [67]:
url = f'http://api.hh.ru/vacancies?employer_id={sber}&page={page}&per_page={num_per_page}&area={Samara}'
res = requests.get(url)
vacancies = res.json()
num_pages = vacancies.get('pages')

In [68]:
all_vacancy_ids = []
for i in tqdm.tqdm(range(num_pages)):
    url = f'http://api.hh.ru/vacancies?employer_id={sber}&page={i}&per_page={num_per_page}&area={Samara}'
    res = requests.get(url)
    vacancies = res.json()
    vacancy_ids = [el.get('id') for el in vacancies.get('items')]
    all_vacancy_ids.extend(vacancy_ids)

100%|██████████| 20/20 [00:08<00:00,  2.40it/s]


In [69]:
today = datetime.datetime.now()
previous_month = today.replace(day=1) - datetime.timedelta(days=1)

all_vacancy_ids = []
interval = 14
for i in tqdm.tqdm(range(num_pages)):
    start_date = previous_month
    interval_end = start_date + datetime.timedelta(days=interval)
    end_date = today

    while start_date <= end_date:
        url = f'http://api.hh.ru/vacancies?employer_id={sber}&page={i}&per_page={num_per_page}&area={Samara}&date_from={start_date.isoformat()}&date_to={interval_end.isoformat()}'
        res = requests.get(url)
        vacancies = res.json()

        vacancy_ids = [el.get('id') for el in vacancies.get('items')]
        all_vacancy_ids.extend(vacancy_ids)

        start_date = interval_end
        interval_end = start_date + datetime.timedelta(days=interval)

100%|██████████| 20/20 [00:05<00:00,  3.86it/s]


In [70]:
len(all_vacancy_ids)

977

In [71]:
#Вытаскиваем все вакансии с hh.ru c запросом полной информации о вакансии по ее id
all_vacancies = []
for vacid in tqdm.tqdm(all_vacancy_ids):
    url = f"https://api.hh.ru/vacancies/{vacid}"
    res = requests.get(url)
    vacancy = res.json()
    all_vacancies.append(vacancy)

100%|██████████| 977/977 [02:26<00:00,  6.67it/s]


In [72]:
#Вытаскиваем все полные описания вакансий
all_vacancy_descriptions = [vac.get("description") for vac in all_vacancies]

#2.	Предообработать текст описаний вакансий (токенизировать, нормировать)

In [73]:
!pip install pymorphy2

In [74]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [75]:
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend([['о'], ['по']])

In [76]:
vac_doc = all_vacancy_descriptions

In [77]:
import re

In [78]:
vac_doc_split = []
for line in vac_doc:
    vac_doc_split.append(' '.join(re.split('[^А-Яа-я]+', line.strip())))

In [79]:
mainland = []
for doc in vac_doc_split:
    new_doc = []
    for word in doc.split():
        if word.lower() not in russian_stopwords:
            new_doc.append(morph.parse(word)[0].normal_form)
    mainland.append(' '.join(new_doc))

#3.	Взять любой текст, описывающий опыт из резюме, и аналогично предобработать

In [81]:
with open('/Users/Roman/Desktop/Python Alex Levchenko/Итоговый проект/Описание опыта из резюме.txt') as f:
    lines = f.readlines()

In [84]:
vac_doc_rezume_split = []
for line in lines:
    vac_doc_rezume_split.append(' '.join(re.split('[^А-Яа-я]+', line.strip())))

In [85]:
island = []
for doc in vac_doc_rezume_split:
    new_doc_rezume = []
    for word in doc.split():
        if word.lower() not in russian_stopwords:
            new_doc_rezume.append(morph.parse(word)[0].normal_form)
    island.append(' '.join(new_doc_rezume))

In [86]:
island

['организация работа коллегиальный орган технический секретарь правление банк замещать секретарь комитет предоставление кредит инвестиция комитет розничный кредитование осуществлять организационно технический обеспечение заседание правление проведение заседание формирование согласование повестка дата время проведение заседание председатель комитет правление приглашение член участник заседание оформление постановление решение протокол заседание обеспечение сохранность документ сдача архив регистрация доведение принять постановление сэодый соответствующий подразделение банк подготовка материал ход выполнение поручение правление справочный информационный работа ранее принять решение постановление правление консультация внутренний клиент вопрос оформление регистрация подготовка документ заседание участие руководство банк контрольный аналитический работа формирование контрольный поручение сэодый постановка поручение контроль учёт информация ход исполнение снятие контроль подготовка отчёт ст

#4.	Посчитать векторное представление описаний вакансий и опыта из резюме

In [87]:
processed_docs = mainland
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
bow_rep_tfidf = tfidf.fit_transform(processed_docs)

In [88]:
bow_rep_tfidf

<977x6004 sparse matrix of type '<class 'numpy.float64'>'
	with 129006 stored elements in Compressed Sparse Row format>

In [89]:
temp = tfidf.transform(island)

In [90]:
print(temp)

  (0, 5852)	0.061834006916283386
  (0, 5740)	0.183839943108628
  (0, 5675)	0.07683106125821453
  (0, 5592)	0.04892518889915877
  (0, 5582)	0.04831006769862931
  (0, 5581)	0.022584399227419227
  (0, 5269)	0.053466772589252594
  (0, 5172)	0.15865253311422442
  (0, 5034)	0.05281086415583195
  (0, 4981)	0.07932626655711221
  (0, 4928)	0.08169966355186106
  (0, 4892)	0.066558181189727
  (0, 4860)	0.03813756327111559
  (0, 4818)	0.08460446671062988
  (0, 4649)	0.18725521894216263
  (0, 4637)	0.09362760947108131
  (0, 4505)	0.04512786133898232
  (0, 4476)	0.05253090404723342
  (0, 4448)	0.04083231117065009
  (0, 4358)	0.11200356176538334
  (0, 4266)	0.07267652079140963
  (0, 4167)	0.04093437729795041
  (0, 4082)	0.043787721395380515
  (0, 3960)	0.062395437482226965
  (0, 3912)	0.08757544279076103
  :	:
  (0, 2040)	0.03699724297993852
  (0, 1964)	0.15865253311422442
  (0, 1963)	0.10123319421716741
  (0, 1959)	0.05713446395257973
  (0, 1935)	0.10799020463226087
  (0, 1853)	0.21430742059964647
 

#5. Посчитать косинусную меру сходства для опыта и описаний вакансий

In [91]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim =  cosine_similarity(bow_rep_tfidf, temp).ravel()

In [92]:
print(cosine_sim)

[0.0051481  0.02186703 0.01644073 0.01717005 0.042383   0.01968088
 0.05701303 0.0113356  0.21331446 0.06959419 0.01541874 0.12574038
 0.03579809 0.09740387 0.03890518 0.03841627 0.02865401 0.03789066
 0.04075766 0.06990924 0.02250641 0.07046454 0.02898744 0.02207265
 0.04496991 0.00870159 0.02839164 0.02684975 0.01001077 0.02547119
 0.01189929 0.03210456 0.0438197  0.14852099 0.01415519 0.03501639
 0.02594366 0.02847529 0.0557913  0.02000303 0.01568958 0.04846157
 0.01871331 0.04630943 0.01488848 0.0197175  0.01752548 0.01122886
 0.01305614 0.00773603 0.01173719 0.00694337 0.00446783 0.0225738
 0.03374175 0.03091168 0.01501402 0.015975   0.04426525 0.03041571
 0.0117383  0.04564173 0.02522808 0.02979021 0.02941163 0.01938159
 0.02510053 0.02011126 0.02566796 0.03755147 0.02953018 0.05379394
 0.04397222 0.01344005 0.01731753 0.02646415 0.03837763 0.05815313
 0.02701541 0.04680478 0.0102076  0.06926336 0.00424361 0.02983973
 0.01406167 0.05924631 0.04797693 0.01778219 0.02422625 0.01842

#6. Ранжировать вакансии по мере их сходства с опытом

In [93]:
ranking = cosine_sim.argsort()

In [94]:
print(ranking)

[584 273 805 888 916 510 138 957 931 282 287 727 651 240 731 672 215 594
 304 375 626 839  82 831  52 939 522 596 509 938   0 641 559 362 521 537
 291 456 976 159 910 853 873 364 108 804 207 230  51 388 562 315 841 280
 567 663 326 292  49 697 848 413 372 710 763 363 547 598 868 544 283 360
  25 450 801 550 269 936 100 284 806 397 351 142 691 560 900 425 299 741
 653 842 908 917 442 183  28 576 796 354 134 164 649 452 324  80 585 531
 937 333 246 179 302 760 307 381 255 268 707 252 786 794 893 501 561 833
 705  47 597 769 169   7 713 186 894 802 838 701 659 285 128  50 441  60
 540 609 675  30 608 417 661 563 107 325 173 669 551 528 217 499 349 414
 391 491 317 256 124 574 684 103 871 357 427  48 642 858 726 406 336 478
 680  73  96 968 832 341 454 768 972 964  97 780 447 102 415 305  84 782
  34 278 133 358 343 504 721 748 211 430 942 955 615 459 811 759 724 930
  44  56 926 518 565 293 667 176 196 111 330 866  10 121 581 136  40 249
 503 213 624 674 711 747 935 469 601 753  57 812 44

#7. Вывести Топ-10 наиболее релевантных

In [95]:
top_10 = cosine_sim.argsort()[-10:]

In [96]:
print((top_10))

[602 746 276 308  11  33 331  90 813   8]


In [97]:
for index in top_10:
    print(index)
    print(vac_doc_split[index])
    print('\n')

602
 Мы разрабатывает продукт Центр управления регионом для руководителей высшего звена субъектов Российской Федерации Губернаторы министры Эта система предназначена для отображения всей необходимой информации для ускорения и облегчения принятия решений В продукт позволяет получать актуальные данные по сферам социально экономического развития региона хода выполнения региональных и национальных проектов статистику и аналитику хода выполнения поручений Президента РФ Правительства РФ Пользователям информация предоставляется в табличном графическом картографическом и текстовом виде в интерактивном режиме Понятная и осязаемая цель Великолепная дружная команда Предлагаем присоединиться к проекту на начальной стадии разработки Удаленная работа из Москвы и Мос области Обязанности Разработка приложений и сервисов Проектирование приложений на Разработка интеграционных сервисов Участие в оценке задач по срокам выполнения декомпозиция задач Проведение Отслеживание исполнения релизного цикла и срок

#8.	Подготовить скрипт принимающий на вход текстовое описание опыта и выводящий Топ-10 наиболее релевантных вакансий из 100 случайно выбранных из выгрузки

In [ ]:
def tokenize_text(text):

    text = ' '.join(
        re.split('[^А-Яа-я]+', text.strip())
    )

    tokenized_text = []
    for word in text.split(' '):
        if word.lower() not in russian_stopwords:
            tokenized_text.append(morph.parse(word)[0].normal_form)
    return tokenized_text

In [64]:
def get_top10_out_of_100(text, vacancies, vectorizer, X):

    # Токенизация (в т.ч. нормализация, морфемизация) текста с опытом
    tokenized_text = tokenize_text(text)

    # Векторизация текста с опытом
    text_vectorized = vectorizer.transform([' '.join(tokenized_text)])

    # Расссет косинусного сходства и выбор 100 случайных текстов
    cosine_total = cosine_similarity(X, text_vectorized).ravel()
    chosen_100_indecies = np.random.randint(0, X.shape[0]+1, size=100)
    cosine_chosen_100 = cosine_total[chosen_100_indecies]

    # Из выбранных 100 текстов выбираем топ 10 (10 наиболее схожих по косинусной метрике)
    top_10_indecies = cosine_chosen_100.argsort()[-10:]
    result = [vacancies[index] for index in top_10_indecies]

    # Функция распечатает топ10 вакансий
    print(*result, sep='\n\n')
    return result